In [0]:
import html5lib

In [0]:
import urllib
import urllib.request
import requests
from bs4 import BeautifulSoup
import os
import cv2
import numpy as np 


pic_num = 1

def store_raw_images(paths, links):
    filename=[]
    global pic_num
    #print(links)
    r = requests.get(links)
    #print(r)
    data = r.text
    #print(data)
    soup = BeautifulSoup(data, "html5lib")
    # In[8]:
    
    for l in soup.find_all('img'):
        #print(l)
        image_urls = l.get("src")
        filename.append(image_urls)
    
        if not os.path.exists(paths):
            os.makedirs(paths)
        #image_urls = str(urllib.request.urlopen(link).read())
        for i in range(len(filename)-90):
            #print(filename[i])
            loadImage(paths,filename[i],pic_num)
            pic_num = pic_num+1
             
        
                    
def loadImage(path,link, counter):
    global pic_num
    if pic_num < counter:
        pic_num = counter+1;
    try:                
        urllib.request.urlretrieve(link, path+"/"+str(counter)+".jpg")
        img = cv2.imread(path+"/"+str(counter)+".jpg")             
        if img is not None:
            cv2.imwrite(path+"/"+str(counter)+".jpg",img)
            #print(counter)

    except Exception as e:
        print(str(e))  


In [0]:
links = 'https://www.shutterstock.com/search/plane'
paths = 'Notvadapav'
store_raw_images(paths,links)

In [0]:
links = 'https://www.shutterstock.com/search/vadapav'
paths = 'vadapav'
store_raw_images(paths,links)

In [0]:
import tensorflow as tf
from skimage import exposure
from tensorflow.contrib.layers import flatten
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.contrib.layers import flatten
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

import matplotlib.pyplot as plt
import glob


In [0]:
def rotateImage(img, angle):
    (rows, cols, ch) = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2), angle, 1)
    return cv2.warpAffine(img, M, (cols,rows))
    
    
def loadBlurImg(path, imgSize):
    img = cv2.imread(path)
    angle = np.random.randint(0, 360)
    img = rotateImage(img, angle)
    img = cv2.blur(img,(5,5))
    img = cv2.resize(img, imgSize)
    return img

def loadImgClass(classPath, classLable, classSize, imgSize):
    x = []
    y = []
    
    for path in classPath:
        img = loadBlurImg(path, imgSize)        
        x.append(img)
        y.append(classLable)
        #print(len(x))
        
    while len(x) < classSize:
        randIdx = np.random.randint(0, len(classPath))
        img = loadBlurImg(classPath[randIdx], imgSize)
        x.append(img)
        y.append(classLable)
        
    return x, y

def loadData(img_size, classSize):
    vadapav = glob.glob('./vadapav/**/*.jpg', recursive=True)
    notvadapav = glob.glob('./Notvadapav/**/*.jpg', recursive=True)
    
    
    imgSize = (img_size, img_size)
    xvadapav, yvadapav = loadImgClass(vadapav, 1, classSize, imgSize)
    xNotvadapav, yNotvadapav = loadImgClass(notvadapav, 0, classSize, imgSize)
    print("There are", len(xvadapav), "vadapav images")
    print("There are", len(xNotvadapav), "not vadapav images")
    
    X = np.array(xvadapav + xNotvadapav)
    y = np.array(yvadapav + yNotvadapav)
    #y = y.reshape(y.shape + (1,))
    return X, y

In [49]:
size = 224
classSize = 500

scaled_X, y = loadData(size, classSize)

n_classes = len(np.unique(y))
print(n_classes)

from keras.utils.np_utils import to_categorical
y = to_categorical(y)

There are 500 vadapav images
There are 500 not vadapav images
2


In [50]:
rand_state = np.random.randint(0, 100)

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

print("Number of classes =", n_classes)
print("train shape X", X_train.shape)
print("train shape y", y_train.shape)

Number of classes = 2
train shape X (800, 224, 224, 3)
train shape y (800, 2)


In [0]:
def kerasModel(inputShape):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape=inputShape, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Conv2D(16, (3, 3), activation='relu', kernel_constraint=maxnorm(3)))
    
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(16, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [54]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

inputShape = (size, size, 3)
model = kerasModel(inputShape)

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_train, y_train, epochs=10)



metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

Epoch 1/10
800/800 [==============================] - 7s 9ms/step - loss: 8.2471 - acc: 0.4788
Epoch 2/10
800/800 [==============================] - 6s 8ms/step - loss: 8.2202 - acc: 0.4900
Epoch 3/10
800/800 [==============================] - 6s 8ms/step - loss: 8.2202 - acc: 0.4900
Epoch 4/10
800/800 [==============================] - 6s 8ms/step - loss: 8.2202 - acc: 0.4900
Epoch 5/10
800/800 [==============================] - 6s 8ms/step - loss: 8.2202 - acc: 0.4900
Epoch 6/10
800/800 [==============================] - 6s 8ms/step - loss: 8.2202 - acc: 0.4900
Epoch 7/10
800/800 [==============================] - 6s 8ms/step - loss: 8.2202 - acc: 0.4900
Epoch 8/10
800/800 [==============================] - 6s 8ms/step - loss: 8.2202 - acc: 0.4900
Epoch 9/10
448/800 [===============>..............] - ETA: 2s - loss: 8.0950 - acc: 0.4978800/800 [==============================] - 6s 8ms/step - loss: 8.2202 - acc: 0.4900
Epoch 10/10
200/200 [==============================] - 1s 5ms/step

# so we get a accuracy of 54% by employing this basic cnn with limited data.